In [19]:
# !pip install gpxpy

In [20]:
import csv, re, os, sys, gpxpy, folium
from datetime import datetime, timedelta, date
import pandas as pd
import numpy as np

dir_path = './'
in_dir_name = 'in_files'
out_dir_name = 'out_files'
time_tolerance = 1000000

In [21]:
def init_dir():
    dir_list = os.listdir(dir_path)
    is_init = False
    if in_dir_name not in dir_list:
        print('作業ディレクトリが存在しません')
        os.mkdir(dir_path + in_dir_name)
        print('make: /' + in_dir_name)
        is_init = True
    if out_dir_name not in dir_list:
        if not is_init:
            print('作業ディレクトリが存在しません')
        os.mkdir(dir_path + out_dir_name)
        print('make: /' + out_dir_name)
        is_init = True
    if is_init:
        print('初期化が完了しました')
        sys.exit(1)

In [22]:
def get_valid_num(pronpt='', get_int=True, max=100000):
    num = 0
    while True:
        if pronpt != '':
            print(pronpt)
        num = input('>')
        try:
            num = float(num)
            if num > max - 1 or num < 0:
                print(f'0以上{max}未満の数を入力してください')
                continue
            else:
                break
        except ValueError:
            print('数字を入力してください')
            continue
    if get_int:
        return int(num)
    else:
        return num

In [23]:
def format_weather_data():
    print('=' * 50)
    print('*** 気象表作成 ***')
    weather_datas = os.listdir(dir_path + in_dir_name)
    weather_datas = [file for file in weather_datas if file.endswith(".csv")]
    if len(weather_datas) == 0:
        print(in_dir_name + 'に .csv ファイルが存在しません')
        sys.exit(2)

    print('使用する気象データファイルを選択')
    for i in range(len(weather_datas)):
        print(f'{i}) {weather_datas[i]}')
    i_filename = weather_datas[get_valid_num(max=len(weather_datas))]
    i_filename = dir_path + in_dir_name + '/' + i_filename
    print()

    with open(i_filename, 'r', encoding='MS932') as csvfile:
        reader = csv.reader(csvfile)

        dl_date = ''
        raw_data = []

        for row_idx, row in enumerate(reader):
            if row_idx == 0:
                dl_date = row[0][row[0].find('：') + 1:]
            if row_idx > 1:
                raw_data.append(row)

    get_md = lambda date: date[date.find('/') + 1:]
    md_start = get_md(raw_data[5][0])  # Month/Day start ex: 3/17
    md_end = get_md(raw_data[-1][0])  # Month/Day end ex: 3/19
    y_start = raw_data[5][0][:raw_data[5][0].find('/')]  # year start
    reference_years = 0
    for row in raw_data:
        if row[0].find(md_start) != -1:
            reference_years += 1
    reference_days = int((len(raw_data) - 5) / reference_years)

    processed_data = {}
    station_data = {}
    invalid_di = []  # invalid date and item
    get_loc = lambda li: raw_data[0][li] + '県' + raw_data[1][li] + '地方'  # li: line_index
    loc = get_loc(1)
    for line_idx in range(1, len(raw_data[0])):
        if get_loc(line_idx) != loc:
            processed_data.setdefault(loc, station_data.copy())
            loc = get_loc(line_idx)
            station_data.clear()
        if raw_data[4][line_idx] == '':
            row = []
            for row_ix in range(5, len(raw_data)):
                if re.match(r'^[-+]?[0-9]*\.?[0-9]+$', raw_data[row_ix][line_idx]):
                    row.append(float(raw_data[row_ix][line_idx]))
                else:
                    row.append(0)
                    invalid_di.append('-'.join([raw_data[row_ix][0], loc, raw_data[2][line_idx]]))
            station_data.setdefault(raw_data[2][line_idx], row)
    processed_data.setdefault(loc, station_data.copy())

    must_items = ['平均気温(℃)', '最高気温(℃)', '最低気温(℃)', '降水量の合計(mm)']
    for mi in must_items:
        if mi not in station_data.keys():
            print(f'{i_filename} には観測項目【{mi}】が含まれていません\n'
                  f'気象データのダウンロードの設定を確認してください')
            sys.exit(2)
    if len(invalid_di) > 0:
        print(f'{i_filename} には欠損データが検出されました\n'
              f'これらのデータを 0 で置換しました\n'
              f'HA, AA, LA, LL の数値が適切に計算できません\n'
              f'検出個所：')
        for invalid in invalid_di:
            print('\t' + invalid)
    else:
        print(f'{i_filename} は適切に読み込まれました')
    print('-' * 50)

    site_infos = []  # site place name and elevation
    obs_infos = []  # observation point elevation
    defs = []

    keys_obs = list(processed_data.keys())  # keys of observation points
    for i in range(1, reference_days + 1):
        print(f'【{i}日目】\n観測地点を選択（半角数字）')
        for j, locs in enumerate(keys_obs):
            print(f'{j}) {locs}')
        obs_choice = get_valid_num(max=len(keys_obs))
        obs_point = keys_obs[obs_choice]
        obs_place_elev = get_valid_num(pronpt=f'{obs_point}の標高を入力', get_int=False)
        obs_infos.append((obs_point, obs_place_elev))

        print('サイト地の名称を入力（ENTERで省略）')
        site_place = input(' >')
        if site_place == '':
            site_place = str(i) + '日目サイト地'
        site_place_elev = get_valid_num(pronpt=f'{site_place}の標高を入力', get_int=False)
        site_infos.append((site_place, site_place_elev))
        defs.append((site_infos[i - 1][1] - obs_infos[i - 1][1]) * -0.006)
        print('-' * 50)

    print('保存ファイル名を入力（ENTERで省略）')
    o_filename = input('>')

    if '降水量の合計(mm)' in station_data:
        value_to_move = station_data['降水量の合計(mm)']
        del station_data['降水量の合計(mm)']
        station_data['降水量の合計(mm)'] = value_to_move

    write_data = []  # write to csv data
    write_data2 = [['', 'HH', 'HA', 'AA', 'LA', 'LL']]
    md_start = datetime.strptime(md_start, '%m/%d')  # convert str to date
    md_end = datetime.strptime(md_end, '%m/%d')
    d = date(datetime.now().year, md_start.month, md_start.day)
    if o_filename == '':
        o_filename = f'{dir_path}{out_dir_name}/{md_start.strftime("%m%d")}-{md_end.strftime("%m%d")}-weatherTable.csv'
    else:
        o_filename = f'{dir_path}{out_dir_name}/{o_filename}.csv'
    for i in range(reference_days):
        write_data.append([d.strftime('%Y/%m/%d') + ' DAY ' + str(i + 1)])
        write_data.append([f'観測地点：{obs_infos[i][0]} 標高：{obs_infos[i][1]}'])
        write_data.append([f'基準地点：{site_infos[i][0]} 標高：{site_infos[i][1]} 補正値：{defs[i]}K'])
        write_data.append([''] + [str(int(y_start) + j) for j in range(reference_years)])
        table = []
        table_crr = []
        sa = [''] * 5  # sum and average HH, HA, AA, LA, HA
        for item in station_data.keys():
            table.append(
                [item] + [processed_data.get(obs_infos[i][0]).get(item)[i + k * reference_days] for k in
                          range(reference_years)])
        for row in table:
            if '℃' in row[0]:
                table_crr = [round(row[elem_idx] + defs[i], 1) for elem_idx in range(1, len(row))]
                write_data.append([row[0]] + table_crr)
            else:
                write_data.append(row)
            if row[0] == '最高気温(℃)':
                sa[0] = str(max(table_crr[1:]))
                sa[1] = str(round(sum(table_crr[1:]) / reference_years, 1))
            elif row[0] == '平均気温(℃)':
                sa[2] = str(round(sum(table_crr[1:]) / reference_years, 1))
            elif row[0] == '最低気温(℃)':
                sa[4] = str(min(table_crr[1:]))
                sa[3] = str(round(sum(table_crr[1:]) / reference_years, 1))
        write_data2.append([d.strftime('%Y/%m/%d')] + sa)
        d += timedelta(days=1)
    write_data.append([])

    with open(o_filename, 'w', newline='', encoding='MS932') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(write_data)
        writer.writerows(write_data2)

    print('\n正常に終了しました')
    print('=' * 50)

In [24]:
# author: https://aotoshiro.jpn.org/2021/2376
def calc_dist(rlat_a, rlat_b, rlon_a, rlon_b):
    A = 6378137.000  # WGS84測地系の楕円体の長半径a
    B = 6356752.314  # WGS84測地系の楕円体の短半径b
    E = np.sqrt((A ** 2 - B ** 2) / A ** 2)  # 離心率
    Dy = rlat_a - rlat_b  # 2点の緯度(latitude)の差 [rad]
    Dx = rlon_a - rlon_b  # 2点の経度(longitude)の差 [rad]
    P = (rlat_a + rlat_b) / 2  # 2点の緯度(latitude)の平均 [rad]
    W = np.sqrt(1 - E ** 2 * np.sin(P) ** 2)
    M = A * (1 - E ** 2) / W ** 3  # 子午線曲率半径
    N = A / W  # 卯酉線曲線半径
    D = np.sqrt((Dy * M) ** 2 + (Dx * N * np.cos(P)) ** 2)
    return D  # 2点間の距離 [m]

In [25]:
def parse_ct(file_path):
    column = ['date', 'time_str', 'time1', 'time2', 'time_passed', 'event', 'elev_spec']
    data_list = []

    prev_datetime = None

    with open(file_path, 'r') as ct_file:
        date_obj = None
        pattern_date_title = r'(\d{4}/\d{1,2}/\d{1,2}.*)'
        pattern_date = r'(\d{4}/\d{1,2}/\d{1,2})'
        pattern_event = r'(\d{1,2}:\d{2},.*)'
        pattern_event_rest = r'(\d{1,2}:\d{2}~\d{1,2}:\d{2},\w+)'

        time_passed = 0

        for line in ct_file:
            parts = line.split(',')
            if len(parts) == 3:
                time_str, event, elev = parts
                elev = elev.strip()
            elif len(parts) == 2:
                time_str, event = parts
                event = event.strip()
                elev = -1019

            if re.match(pattern_date_title, line):
                date_str = re.match(pattern_date, line)[0]
                date_obj = datetime.strptime(date_str, '%Y/%m/%d')

            elif re.match(pattern_event, line):
                time_obj = datetime.strptime(time_str, '%H:%M')
                date_obj = datetime(date_obj.year, date_obj.month, date_obj.day, time_obj.hour, time_obj.minute)
                if prev_datetime is not None:
                    time_passed = (date_obj - prev_datetime).total_seconds() / 60
                data_list.append([date_obj, time_str, time_str, None, time_passed, event, elev])
                prev_datetime = date_obj

            elif re.match(pattern_event_rest, line):
                time1, time2 = time_str.split('~')
                time_obj = datetime.strptime(time1, '%H:%M')
                date_obj = datetime(date_obj.year, date_obj.month, date_obj.day, time_obj.hour, time_obj.minute)
                time2_obj = datetime.strptime(time2, '%H:%M')
                if prev_datetime is not None:
                    time_passed = (date_obj - prev_datetime).total_seconds() / 60
                data_list.append([date_obj, time_str, time1, time2, time_passed, event, elev])
                prev_datetime = datetime(date_obj.year, date_obj.month, date_obj.day, time2_obj.hour, time2_obj.minute)

    time_event_df = pd.DataFrame(data_list, columns=column)

    return time_event_df


def correct_ct(time_event_df, new_dir_name):
    crt_ct_file_path = f'{dir_path}{out_dir_name}/{new_dir_name}/courseTime_{time_event_df.loc[0, "date"].date()}-{time_event_df.loc[len(time_event_df) - 1, "date"].date()}.txt'
    with open(crt_ct_file_path, 'w', encoding='shift-jis') as ccf:  # ccf: correct_courseTime_file
        prev_datetime = datetime(1910, 3, 17, 0, 0)
        for index, row in time_event_df.iterrows():
            if prev_datetime.date() != row['date'].date():
                ccf.write(f'\n{row["date"].strftime("%Y/%m/%d")}コースタイム\n')
            line_str = f'{row["time_str"]},{row["event"].strip()},{round(int(row["elev_calc"]), -1)}'
            if row['elev_spec'] != -1019:
                line_str = f'{line_str}({row["elev_spec"]})'
            ccf.write(line_str + '\n')
            prev_datetime = row['date']


def analyze_ct(time_event_df, new_dir_name):
    analytics_file_path = f'{dir_path}{out_dir_name}/{new_dir_name}/courseTimeAnalytics_{time_event_df.loc[0, "date"].date()}-{time_event_df.loc[len(time_event_df) - 1, "date"].date()}.csv'
    to_csv_df = time_event_df[
        ['date', 'time_str', 'time_passed', 'event', 'elev_spec', 'elev_calc', 'lat', 'lon', 'dist']]
    to_csv_df.to_csv(analytics_file_path, index=True, encoding='utf-8')
    return 0


class GPXDataAnalyzer:
    def __init__(self, gpx_file_path):
        self.gpx_file_path = gpx_file_path
        self.gpx_data = self.load_gpx_data()

    def load_gpx_data(self):
        with open(self.gpx_file_path, 'r', encoding='utf-8') as gpx_file:
            gpx = gpxpy.parse(gpx_file)
            return gpx

    # def linear_interpolation(self, point1, point2, target_time):
    #     time1 = point1.time.timestamp()
    #     time2 = point2.time.timestamp()
    #     value1 = point1.elevation  # 例: 高度データ
    #     value2 = point2.elevation  # 例: 高度データ
    #
    #     interpolated_value = value1 + (value2 - value1) * ((target_time - time1) / (time2 - time1))
    #     return interpolated_value

    def search_gpx_data_by_time(self, target_time_jst, time_tolerance):
        closest_point = None
        closest_time_diff = None

        jst_offset = timedelta(hours=9)  # JSTとUTCの固定オフセット
        target_time_utc = target_time_jst - jst_offset  # JSTからUTCに変換
        for track in self.gpx_data.tracks:
            for segment in track.segments:
                for point in segment.points:
                    if point.time:
                        time_diff = abs((point.time.replace(tzinfo=None) - target_time_utc).total_seconds())
                        if closest_point is None or time_diff < closest_time_diff:
                            closest_point = point
                            closest_time_diff = time_diff

        if closest_point and closest_time_diff <= time_tolerance:
            return closest_point
        else:
            return None


def plot_ct():
    print('=' * 50)
    print('*** コースタイム・軌跡分析 ***')

    in_datas = os.listdir(dir_path + '/' + in_dir_name)
    gpx_datas = [file for file in in_datas if file.endswith(".gpx")]
    ct_datas = [file for file in in_datas if file.endswith(".txt")]
    if len(gpx_datas) == 0:
        print(in_dir_name + 'に .gpx ファイルが存在しません')
        sys.exit(3)
    if len(ct_datas) == 0:
        print(in_dir_name + 'に .txt ファイルが存在しません')
        sys.exit(3)

    print('使用するGPXデータファイルを選択')
    for i in range(len(gpx_datas)):
        print(f'{i}) {gpx_datas[i]}')
    gpx_filename = gpx_datas[get_valid_num(max=len(gpx_datas))]
    gpx_file_path = dir_path + in_dir_name + '/' + gpx_filename
    print('-' * 50)
    print('使用するコースタイムファイルを選択')
    for i in range(len(ct_datas)):
        print(f'{i}) {ct_datas[i]}')
    ct_filename = ct_datas[get_valid_num(max=len(ct_datas))]
    ct_file_path = dir_path + in_dir_name + '/' + ct_filename
    print('-' * 50)

    time_event_df = parse_ct(ct_file_path)
    print(f'{ct_file_path} は適切に読み込まれました')
    analyzer = GPXDataAnalyzer(gpx_file_path)
    print(f'{gpx_file_path} は適切に読み込まれました')
    print('-' * 50)

    lat_list = []
    lon_list = []
    elev_list = []
    for event in time_event_df['date']:
        closest_point = analyzer.search_gpx_data_by_time(event, time_tolerance)
        lat_list.append(closest_point.latitude)
        lon_list.append(closest_point.longitude)
        elev_list.append(closest_point.elevation)
    time_event_df['elev_calc'] = elev_list
    time_event_df['lat'] = lat_list
    time_event_df['lon'] = lon_list

    dist = []
    rlat_a, rlon_a = 0, 0
    for index, row in time_event_df.iterrows():
        if index == 0:
            dist.append(0)
            rlat_a = np.radians(row['lat'])
            rlon_a = np.radians(row['lon'])
        else:
            dist.append(calc_dist(rlat_a, np.radians(row['lat']), rlon_a, np.radians(row['lon'])))
            rlat_a = np.radians(row['lat'])
            rlon_a = np.radians(row['lon'])
    time_event_df['dist'] = dist

    # pd.set_option('display.max_rows', None)
    # pd.set_option('display.max_columns', None)
    # print(time_event_df)

    map_center = [time_event_df.loc[0, 'lat'], time_event_df.loc[0, 'lon']]
    gpx_map = folium.Map(tiles='http://cyberjapandata.gsi.go.jp/xyz/std/{z}/{x}/{y}.png', attr='© GSI Japan',
                         location=map_center, zoom_start=15)

    for track in analyzer.gpx_data.tracks:
        for segment in track.segments:
            folium.PolyLine(
                locations=[(point.latitude, point.longitude) for point in segment.points],
                color='blue'
            ).add_to(gpx_map)

    for index, row in time_event_df.iterrows():
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            tooltip=f'{row["date"].strftime("%m/%d")} {row["time_str"]} {row["event"].strip()}: {round(int(row["elev_calc"]), -1)}m',
            icon=folium.Icon(color='blue')
        )
        marker.add_to(gpx_map)

    new_dir_name = f'{time_event_df.loc[0, "date"].date()}-{time_event_df.loc[len(time_event_df) - 1, "date"].date()}'
    if new_dir_name not in os.listdir(dir_path + out_dir_name):
        os.mkdir(f'{dir_path}{out_dir_name}/{new_dir_name}')

    map_file_path = f'{dir_path}{out_dir_name}/{new_dir_name}/courseTimeMap_{time_event_df.loc[0, "date"].date()}-{time_event_df.loc[len(time_event_df) - 1, "date"].date()}.html'
    gpx_map.save(map_file_path)
    correct_ct(time_event_df, new_dir_name)
    analyze_ct(time_event_df, new_dir_name)

    print(f'{dir_path}{out_dir_name}/{new_dir_name} に結果を保存しました\n')
    print('正常に終了しました')
    print('=' * 50)

In [26]:
def parce_gpx(gpx_file_path):
    column = ['day', 'lat', 'lon', 'name', 'd_h', 'd_t', 'd_dist', 'h', 't']
    data_list = []
    d_t_max = 36000  # 10h

    gpx_data = None
    with open(gpx_file_path, 'r', encoding='utf-8') as gpx_file:
        gpx_data = gpxpy.parse(gpx_file)

    is_1st = True
    prev_point = None
    prev_date = None
    rlat_a, rlon_a = 0, 0
    day = 1
    for track in gpx_data.tracks:
        for segment in track.segments:
            for point in segment.points:
                jst_date = point.time + timedelta(hours=9)
                if is_1st:
                    data_list.append(
                        [day, point.latitude, point.longitude, point.name, 0, timedelta(0), 0, point.elevation,
                         jst_date])
                    is_1st = False
                else:
                    d_h = point.elevation - prev_point.elevation
                    d_t = jst_date - prev_date
                    d_dist = calc_dist(rlat_a, np.radians(point.latitude), rlon_a, np.radians(point.longitude))
                    if d_t.total_seconds() > d_t_max:
                        day += 1
                        d_t = timedelta(0)
                    data_list.append(
                        [day, point.latitude, point.longitude, point.name, d_h, d_t, d_dist, point.elevation, jst_date])

                prev_point = point
                prev_date = jst_date
                rlat_a, rlon_a = np.radians(point.latitude), np.radians(point.longitude)

    gpx_df = pd.DataFrame(data_list, columns=column)

    # pd.set_option('display.max_rows', None)
    # pd.set_option('display.max_columns', None)
    # print(gpx_df)

    return gpx_df


def sum_each_delta(gpx_df):
    column = ['day', 'lat', 'lon', 'name', 'd_up', 'd_down', 'd_t', 'd_dist', 't', 'h']
    data_list = []

    get_1st_pwn = False  # get 1st point with name
    d_up, d_down, d_t, d_dist = 0, 0, timedelta(0), 0
    for index, row in gpx_df.iterrows():
        if not get_1st_pwn and not row['name'] is None:
            data_list.append(
                [row['day'], row['lat'], row['lon'], row['name'], 0, 0, timedelta(0), 0, row['t'], row['h']])
            get_1st_pwn = True
        else:
            if row['d_h'] > 0:
                d_up += row['d_h']
            else:
                d_down -= row['d_h']
            d_t = d_t + row['d_t']
            d_dist += row['d_dist']

            if not row['name'] is None:
                data_list.append(
                    [row['day'], row['lat'], row['lon'], row['name'], d_up, d_down, d_t, d_dist, row['t'], row['h']])
                d_up, d_down, d_t, d_dist = 0, 0, timedelta(0), 0

    name_point_df = pd.DataFrame(data_list, columns=column)

    # pd.set_option('display.max_rows', None)
    # pd.set_option('display.max_columns', None)
    # print(name_point_df)
    # name_point_df.to_csv('output.csv', index=False, encoding='MS932')

    return name_point_df


def print_ct(name_point_df):
    day = 1
    is_1st = True
    print_list = []
    d_sum_dist, d_sum_t, d_sum_up, d_sum_down = 0, timedelta(0), 0, 0
    for index, row in name_point_df.iterrows():
        time = row["t"].strftime("%m/%d")
        d_t = int(row["d_t"].total_seconds() / 60)
        name = re.sub(r'\s*\[.*\]\s*', '', row['name'])
        if is_1st:
            print_list.append(f'【{time} day{day}】')
            print_list.append(f'{name} ({int(row["h"])}m)')
            is_1st = False
        else:
            if not row['day'] == day or index == len(name_point_df) - 1:
                last_point = print_list[-1]

                total_seconds = d_sum_t.total_seconds()
                hours, seconds = divmod(total_seconds, 3600)
                minutes, seconds = divmod(seconds, 60)
                formatted_time = f"{int(hours)}時間{int(minutes)}分"
                print_list.append(
                    f'\n実動距離：{int(d_sum_dist / 100) / 10}km 実動時間：{formatted_time} 行動時間：時間分')
                print_list.append(f'▲{d_sum_up}▼{d_sum_down}')

                if not index == len(name_point_df) - 1:
                    day = row['day']
                    print_list.append(f'\n\n【{time} day{day}】')
                    print_list.append(last_point)
                    print_list.append(
                        f'\n ↓ {int(row["d_dist"])}m/{d_t}min ▲{int(row["d_up"])}▼{int(row["d_down"])}')
                    print_list.append(f'\n{name} ({int(row["h"])}m)')
                    d_sum_dist, d_sum_t, d_sum_up, d_sum_down = 0, timedelta(0), 0, 0
            else:
                d_sum_dist += row['d_dist']
                d_sum_t += row['d_t']
                d_sum_up += row['d_up']
                d_sum_down += row['d_down']
                print_list.append(
                    f'\n ↓ {int(row["d_dist"])}m/{d_t}min ▲{int(row["d_up"])}▼{int(row["d_down"])}')
                print_list.append(f'\n{name} ({int(row["h"])}m)')

    out_file_path = f'{dir_path}{out_dir_name}/{datetime.now().strftime("%Y%m%d %H-%M-%S")}.txt'
    with open(out_file_path, "w") as file:
        for item in print_list:
            file.write(item + "\n")
    print(f'{out_file_path} に結果を保存しました\n')


# ヤマレコでルートを計画すると経由地点の名称入りgpxファイルが得られる。これを利用して企画書用のコースタイムを一括で作成できるプログラムを組もうかと思ったが、山頂などの標高誤差が大きすぎて実用的でないため中止。残念でしたー
def make_ct_string():
    print('=' * 50)
    print('*** 企画書用コースタイム作成 ***')

    in_datas = os.listdir(dir_path + '/' + in_dir_name)
    gpx_datas = [file for file in in_datas if file.endswith(".gpx")]
    if len(gpx_datas) == 0:
        print(in_dir_name + 'に .gpx ファイルが存在しません')
        sys.exit(4)

    print('使用するGPXデータファイルを選択')
    for i in range(len(gpx_datas)):
        print(f'{i}) {gpx_datas[i]}')
    gpx_filename = gpx_datas[get_valid_num(max=len(gpx_datas))]
    gpx_file_path = dir_path + in_dir_name + '/' + gpx_filename
    print('-' * 50)

    gpx_df = parce_gpx(gpx_file_path)
    print(f'{gpx_file_path} は適切に読み込まれました')
    print('-' * 50)

    name_point_df = sum_each_delta(gpx_df)
    print_ct(name_point_df)

    print('正常に終了しました')
    print('=' * 50)

In [27]:
try:
    init_dir()

    print('使用する機能を選択')
    features = [
        '気象表作成',
        'コースタイム・軌跡分析',
        '企画書用コースタイム作成 (β)'
    ]
    for i in range(len(features)):
        print(f'{i}) {features[i]}')
    feature_choice = get_valid_num(pronpt='数字を入力', max=len(features))

    if feature_choice == 0:
        format_weather_data()
    elif feature_choice == 1:
        plot_ct()
    elif feature_choice == 2:
        make_ct_string()
except SystemExit:
    print('\nプログラムを終了します')

    print('=' * 50)

使用する機能を選択
0) 気象表作成
1) コースタイム・軌跡分析
2) 企画書用コースタイム作成 (β)
数字を入力
*** 気象表作成 ***
使用する気象データファイルを選択
0) data (13).csv
1) data (8).csv

./in_files/data (8).csv は適切に読み込まれました
--------------------------------------------------
【1日目】
観測地点を選択（半角数字）
0) 福島県桧枝岐地方
0以上1未満の数を入力してください
0以上1未満の数を入力してください
0以上1未満の数を入力してください
福島県桧枝岐地方の標高を入力
サイト地の名称を入力（ENTERで省略）
0の標高を入力
--------------------------------------------------
保存ファイル名を入力（ENTERで省略）

正常に終了しました
